<a href="https://colab.research.google.com/github/andrav15/Regresor-dynamic/blob/main/Regresor_dynamic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import torch
import math

In [2]:
class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate five parameters and assign them as members.
        """
        super().__init__()#cls parinte
        self.a = torch.nn.Parameter(torch.randn(()))#genereaza val rando dintr-o distributie normala standard(media 0, deviatia standard 1)
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):#modul in care modelul va produce pred pe baza lui x
        """
        For the forward pass of the model, we randomly choose either 4, 5
        and reuse the e parameter to compute the contribution of these orders.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same parameter many
        times when defining a computational graph.
        """
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3#fct polinomiala
        for exp in range(4, random.randint(4, 6)):#determina rando daca sa se includa trmenul suplimentar de grad 4 sau 5
            y = y + self.e * x ** exp#aduce o dinamica supl modelului->adaptare mai buna la datele de antrenare
        return y

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'

In [3]:
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

In [4]:
model = DynamicNet()

In [5]:
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
#SGD:alg de optimizare utilizat pt minimzarea fct de pierdere->actual param prin efectuarea de pasi mici in directia negativa a grad
#momentum ajuta la accelerarea SGD in directia relevanta si atenueaza oscilatiile.
#tine evidenta grad de mai sus si ajuta la netezirea actualizarilor.
#momentum=.9=>90% din actualizarea anterioara influenteaza actualizarea curenta

In [6]:
for t in range(30000):#30000 de epoci
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)#ia tensorul de intrare x->iesire y_pred predictia modelului

    # Compute and print loss
    loss = criterion(y_pred, y)#compara predictia modelului cu cea reala->calculul pierderii
    if t % 2000 == 1999:#verif daca epoca curenta este la fiecare 2000 de iterarii; true->print nr epocii curente si pierderea
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()#resetam grad
    loss.backward()#backpropagation=>calc gradientelor pierderii in raport cu param modelului
    optimizer.step()#actualizarea greutatilor

print(f'Result: {model.string()}')

1999 4641.08349609375
3999 2232.379638671875
5999 1080.21533203125
7999 499.7969665527344
9999 257.72613525390625
11999 130.2945098876953
13999 67.63865661621094
15999 36.90419006347656
17999 22.710865020751953
19999 15.393253326416016
21999 12.1082124710083
23999 10.381576538085938
25999 9.59962272644043
27999 9.208240509033203
29999 9.014098167419434
Result: y = -0.013405775651335716 + 0.8523516654968262 x + 0.001866192207671702 x^2 + -0.09291037917137146 x^3 + 7.302005542442203e-05 x^4 ? + 7.302005542442203e-05 x^5 ?
